In [2]:
import pandas as pd

In [7]:
df = pd.read_csv('/content/enrol_anomalies_false.csv')

In [8]:
df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,02-03-2025,meghalaya,east khasi hills,793121,11,61,37
1,09-03-2025,maharashtra,aurangabad,431001,42,46,12
2,09-03-2025,delhi,west delhi,110018,29,11,15
3,09-03-2025,delhi,west delhi,110059,93,42,42
4,09-03-2025,west bengal,coochbehar,736135,19,12,19


In [9]:
df.shape

(120421, 7)

In [6]:
pip install rapidfuzz jellyfish

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 27.8 MB/s eta 0:00:00


In [10]:
## new

import json
import numpy as np
import pandas as pd
from rapidfuzz import process, fuzz
import jellyfish

with open("/content/state_district_reference.json", "r") as f:
    ref = json.load(f)

state_districts = {
    state: data["districts"]
    for state, data in ref.items()
}

def enhanced_similarity(a, b, **kwargs):
    return max(
        fuzz.token_set_ratio(a, b),
        fuzz.partial_ratio(a, b),
        fuzz.ratio(a, b),
        jellyfish.jaro_winkler_similarity(a, b) * 100
    )

def token_or_substring_ok(a, b):
    a = a.lower()
    b = b.lower()

    a_tokens = a.split()
    b_tokens = b.split()

    # 1️⃣ token overlap (multi-word safety)
    if set(a_tokens) & set(b_tokens):
        return True

    # 2️⃣ substring
    if a in b or b in a:
        return True

    # 3️⃣ single-word prefix rejection
    if len(a_tokens) == 1 and len(b_tokens) == 1:
        if a[:4] != b[:4]:
            return False

    # 4️⃣ character overlap
    a_chars = set(a)
    b_chars = set(b)
    overlap_ratio = len(a_chars & b_chars) / min(len(a_chars), len(b_chars))

    return overlap_ratio >= 0.6



def best_district_match(state, district, threshold=80, min_gap=12):
    if pd.isna(state) or pd.isna(district):
        return np.nan

    if state not in state_districts:
        return np.nan

    matches = process.extract(
        district,
        state_districts[state],
        scorer=enhanced_similarity,
        limit=2
    )

    if not matches:
        return np.nan

    best_name, best_score, _ = matches[0]

    if best_score < threshold:
        return np.nan

    if len(matches) > 1:
        if (best_score - matches[1][1]) < min_gap:
            return np.nan

    if not token_or_substring_ok(district, best_name):
        return np.nan

    return best_name



df["district_new"] = df.apply(
    lambda x: best_district_match(x["state"], x["district"]),
    axis=1
)


In [11]:
df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,district_new
0,02-03-2025,meghalaya,east khasi hills,793121,11,61,37,NaN
1,09-03-2025,maharashtra,aurangabad,431001,42,46,12,NaN
2,09-03-2025,delhi,west delhi,110018,29,11,15,west
3,09-03-2025,delhi,west delhi,110059,93,42,42,west
4,09-03-2025,west bengal,coochbehar,736135,19,12,19,cooch behar


In [12]:
#Saving Cleaned Data
df_cleaned = df[~df['district_new'].isna()]

In [13]:
df_cleaned.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,district_new
2,09-03-2025,delhi,west delhi,110018,29,11,15,west
3,09-03-2025,delhi,west delhi,110059,93,42,42,west
4,09-03-2025,west bengal,coochbehar,736135,19,12,19,cooch behar
5,15-03-2025,delhi,west delhi,110059,24,34,18,west
10,20-03-2025,andhra pradesh,spsr nellore,524311,15,19,17,sri potti sriramulu nellore


In [14]:
df_cleaned['district'] = df_cleaned['district_new']

/tmp/ipython-input-1763447331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['district'] = df_cleaned['district_new']


In [15]:
df_cleaned.drop(columns=['district_new'], inplace=True)


/tmp/ipython-input-3414518825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop(columns=['district_new'], inplace=True)


In [16]:
df_cleaned.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
2,09-03-2025,delhi,west,110018,29,11,15
3,09-03-2025,delhi,west,110059,93,42,42
4,09-03-2025,west bengal,cooch behar,736135,19,12,19
5,15-03-2025,delhi,west,110059,24,34,18
10,20-03-2025,andhra pradesh,sri potti sriramulu nellore,524311,15,19,17


In [17]:
df_cleaned.shape

(38627, 7)

In [18]:
df_cleaned.to_csv('enrol_anomalies_anomalies_clean.csv',index=False)

In [19]:
# Saving False Data
df_false = df[df['district_new'].isna()]

In [20]:
df_false.shape

(81794, 8)

In [21]:
df_false.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,district_new
0,02-03-2025,meghalaya,east khasi hills,793121,11,61,37,NaN
1,09-03-2025,maharashtra,aurangabad,431001,42,46,12,NaN
6,15-03-2025,meghalaya,west khasi hills,793119,34,10,148,NaN
7,15-03-2025,delhi,north west delhi,110086,35,15,10,NaN
8,15-03-2025,meghalaya,west jaintia hills,793150,92,17,30,NaN


In [22]:
#simple check
df_false[~df_false['district_new'].isna()]

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,district_new


In [24]:
df_false.to_csv('enrol_anomalies_anomalies_false.csv',index=False)